In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual

#Properties of the fluid
K = 0.001                           #Pipe roughness [mm]
n = 1.31*10**-6                     #Viscosity [m2/s]
g = 9.81                            #Gravity [m2/s]
E = 784532117.25*100                #Pipe's elasticity [Pa]
Kf = 1.96*10**9                     #Fluid's elasticity [Pa]
ρ = 999.4                           #Fluid's density [kg/m3]
μ = 0.3                             #Pipe's freedom

#Initial conditions
elevd = 10
elevup = 30
Q = 1.988
D = 0.77
e = 0.01
l = 1000
hfl = 10                   #Percentage of local losses [%]

#Calculation conditions
nodes = 11
tclose = 0.5
hzero = 120
tmax = 50

Din = D - 2 * e 
A = (np.pi * Din ** 2) * 0.25
v = Q/A

def fcw(n, D, v, K):
    midf = []
    Re = v * D / n
    f = (1/(2*np.log10(K/(3.72*D*1000))))**2
    midf.append(f)
    for i in range(1, 10):
        midf.append((1/(2*np.log10((K/(3.72*D*1000)) + (2.51/(Re*np.sqrt(midf[i - 1]))))))**2)
    f = midf[-1]
    return f

f = fcw(n, Din, v, K)

hf = f*l*v**2/(Din*2*g)
hl = hfl/100*hf
dh = hf + hl

a = np.sqrt(Kf/ρ)/(np.sqrt(1 + (Kf * D * (1 - μ**2))/(E * e)))
dx = l/(nodes - 1)
dt = dx/a
delel = (elevup - elevd)/(nodes - 1)
ca = (g * np.pi * D**2)/(4 * a)
step = np.arange(0, tmax + dt, dt)

x = np.zeros(nodes)
h = np.zeros(nodes)
hlow = np.zeros(nodes)
hhigh = np.zeros(nodes)
pipez = np.zeros(nodes)
head = np.zeros(nodes)

for i in range(nodes):
    x[i] = i * dx
    h[i] = hzero - dh * i / nodes
    hlow[i] = h[i]
    hhigh[i] = h[i]
    pipez[i] = elevup - delel * i
    head[i] = h[i] - pipez[i]

Qic = np.ones(nodes) * Q
Vic = np.ones(nodes) * v
Hic = h

def chars(nodes, Q, D, n, K, H, ca, dt, hzero, V0, t, tclose, A):
    def fcw(n, D, v, K):
        midf = []
        Re = v * D / n
        f = (1/(2*np.log10(K/(3.72 * D * 1000))))**2
        midf.append(f)
        for i in range(1, 10):
            midf.append((1 / (2 * np.log10((K / (3.72 * D * 1000)) + (2.51/(Re * np.sqrt(midf[i - 1]))))))**2)
        f = midf[-1]
        return f
    cm_ = np.zeros(nodes)
    cn_ = np.zeros(nodes)
    Hnew = np.zeros(nodes)
    Qnew = np.zeros(nodes)
    Vnew = np.zeros(nodes)
    for i in range(1, nodes):
        if Q[i - 1] != 0:
            vi = Q[i - 1] / A
            f = fcw(n, D, np.abs(vi), K)
        else:
            f = 0
        cfa = f * dt / (np.pi * D ** 3/2)
        cm = Q[i - 1] + ca * H[i - 1] - cfa * Q[i - 1] * abs(Q[i - 1])
        cm_[i] = cm
    for i in range(0, nodes - 1):
        if Q[i + 1] != 0:
            vi = Q[i + 1] / A
            f = fcw(n, D, np.abs(vi), K)
        else:
            f = 0
        cfb = f * dt / (np.pi * D ** 3/2)
        cn = Q[i + 1] - ca * H[i + 1] - cfb * Q[i + 1] * abs(Q[i + 1])
        cn_[i] = cn
    Hnew = 0.5 * (cm_ - cn_) / ca
    Hnew[0] = hzero    
    Qnew = cn_ + ca * Hnew
    Vnew = 4 * Qnew / (np.pi * D ** 2)
    Hnew = 0.5 * (cm_ - cn_) / ca
    Hnew[0] = hzero
    if t > tclose:
        Vnew[-1] = 0
    else:
        Vnew[-1] = V0 - t / tclose * V0
    Qnew[-1] = Vnew[-1] * np.pi * D ** 2 / 4
    Hnew[-1] = (cm_[-1] - Qnew[-1]) / ca
    return Hnew, Qnew, Vnew

H = []
Q = []
V = []

H.append(Hic)
Q.append(Qic)
V.append(Vic)

H.append(chars(nodes, Qic, Din, n, K, Hic, ca, dt, hzero, v, dt, tclose, A)[0])
Q.append(chars(nodes, Qic, Din, n, K, Hic, ca, dt, hzero, v, dt, tclose, A)[1])
V.append(chars(nodes, Qic, Din, n, K, Hic, ca, dt, hzero, v, dt, tclose, A)[2])

for i in range(1, len(step)):
    H.append(chars(nodes, Q[i], Din, n, K, H[i], ca, dt, hzero, v, step[i], tclose, A)[0])
    Q.append(chars(nodes, Q[i], Din, n, K, H[i], ca, dt, hzero, v, step[i], tclose, A)[1])
    V.append(chars(nodes, Q[i], Din, n, K, H[i], ca, dt, hzero, v, step[i], tclose, A)[2])

Head = pd.DataFrame(H)
Discharge = pd.DataFrame(Q)
Velocity = pd.DataFrame(V)

Head.head(10)

,0,1,2,3,4,5,6,7,8,9,10
0,120.0,118.621685,117.243370,115.865056,114.486741,113.108426,111.730111,110.351796,108.973482,107.595167,106.216852
1,120.0,118.621685,117.243370,115.865056,114.486741,113.108426,111.730111,110.351796,108.973482,107.595167,193.324948
2,120.0,118.621685,117.243370,115.865056,114.486741,113.108426,111.730111,110.351796,108.973482,194.446477,193.395155
3,120.0,118.621685,117.243370,115.865056,114.486741,113.108426,111.730111,110.351796,195.568628,194.516453,281.014654
4,120.0,118.621685,117.243370,115.865056,114.486741,113.108426,111.730111,196.691402,195.638375,281.938475,368.121826
5,120.0,118.621685,117.243370,115.865056,114.486741,113.108426,197.814797,196.760919,282.863066,368.911919,456.132585
6,120.0,118.621685,117.243370,115.865056,114.486741,198.938813,197.884086,283.788428,369.702544,456.857592,475.791178
7,120.0,118.621685,117.243370,115.865056,200.063450,199.007876,284.714561,370.493706,457.582581,476.513142,476.891805
8,120.0,118.621685,117.243370,201.188707,200.132286,285.641467,371.285409,458.307559,477.234889,477.613322,477.235087
9,120.0,118.621685,202.314583,201.257318,286.569148,372.077656,459.032534,477.956432,478.334619,477.956803,478.334821


In [14]:
time = Head.index * dt
def plot_hammer(i = 0):
    plt.figure(figsize = (15, 5))
    plt.plot(time, Head[i], 'b', zorder = 11)
    plt.vlines(tclose, -abs(Head[10].min()), Head[10].max(), 'r', ls = '--', alpha = 0.5, zorder = 10)
    plt.hlines(pipez[i], 0, time[-1], 'darkgreen', alpha = 0.5, zorder = 8)
    plt.hlines(0, -5, time[-1], 'black', zorder = 9)
    plt.vlines(-5, -abs(Head[10].min()), Head[10].max(), 'black', zorder = 8)
    plt.legend(['Pressure wave', 'Valve close {} secs'.format(tclose), 'Ground level'])
    plt.title('Node {}'.format(i))
    plt.xlabel('Time [sec]')
    plt.ylabel('Head [m]')
    plt.grid();

interact(plot_hammer, i = (0, (nodes - 1), 1));

interactive(children=(IntSlider(value=0, description='i', max=10), Output()), _dom_classes=('widget-interact',…

In [15]:
def plot_hammer_line(i = 0):
    plt.figure(figsize = (15, 5))
    plt.plot(Head.loc[i], 'b', zorder = 10)
    plt.plot(pipez, 'darkgreen', alpha = 0.8, zorder = 10)
    plt.plot(Head.max(axis = 0), 'r--')
    plt.plot(Head.min(axis = 0), 'y--')
    plt.hlines(0, 0, nodes - 1, 'black', zorder = 9)
    plt.vlines(0, -abs(Head[10].min()), Head[10].max(), 'black', zorder = 8)
    plt.legend(['Pressure wave', 'Ground level', 'Maximum', 'Minimum'])
    plt.title('Time {:.3f} sec'.format(i * dt))
    plt.xlabel('Nodes [-]')
    plt.ylabel('Head [m]')
    plt.grid();
    
interact(plot_hammer_line, i = (0, len(step), 1));

interactive(children=(IntSlider(value=0, description='i', max=424), Output()), _dom_classes=('widget-interact'…